In [ ]:
#----------------------------------------------------------------------------------------------------------------------------------#
# Use this only if you have to reprogram the arduino. This will prorgam the .ino file found in the 'arduino_sketchbook' folder
# to the attached arduino via the USB UART.
#----------------------------------------------------------------------------------------------------------------------------------#

%cd arduino_sketchbook_leading_car
! make clean upload;
%cd ..

In [ ]:
#----------------------------------------------------------------------------------------------------------------------------------#
# Loading necessary libraries and their instantiations. Make sure that the Camera instantation happens only once or it ends up 
# breaking the notebook and would require a restart. We use pygame to emulate a display even though there isn't one :E.
#----------------------------------------------------------------------------------------------------------------------------------#

import warnings
warnings.filterwarnings('ignore')

import traitlets
from IPython.display import display

from time import sleep
import threading 
import os

import cv2

import pygame
import pygame.display

os.environ["SDL_VIDEODRIVER"] = "dummy"
pygame.display.init()
screen = pygame.display.set_mode((1,1))

pygame.init()
clock = pygame.time.Clock()
pygame.joystick.init()

import serial
ser = serial.Serial('/dev/ttyACM0', 57600, timeout = 0.5)

from misc_scripts.rpi_camera import CSICamera
camera = CSICamera(width=300, height=300)

import ipywidgets.widgets as widgets
image_widget = widgets.Image(format='jpeg', width=300, height=300)

print('Primary initializations complete.')

In [ ]:
#----------------------------------------------------------------------------------------------------------------------------------#
# Send Throttle and Steering values for the servos to the arduino connected via the USB UART serial.
#----------------------------------------------------------------------------------------------------------------------------------#
def serial_writer(val1, val2):
    val1 = format(val1,"03d")
    val2 = format(val2,"03d")
    val = val1 + ',' + val2 + '*'
    ser.write(str(val).encode())

#----------------------------------------------------------------------------------------------------------------------------------#
# Follows from the arduino 'map' funciton used to translate one range of values to another range. 
#----------------------------------------------------------------------------------------------------------------------------------#
def constrain(val, min_val, max_val):
    return min(max_val, max(min_val, val))

def mapper(x, in_min, in_max, out_min, out_max):
    return int((x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min)   

#----------------------------------------------------------------------------------------------------------------------------------#
# Converts image to a bytes format for display with a Jupyter Image widget.
#----------------------------------------------------------------------------------------------------------------------------------#
def bgr8_to_jpeg(value, quality = 75):
    return bytes(cv2.imencode('.jpg', value)[1])

In [ ]:
camera.running = True
camera1_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

In [ ]:
camera1_link.link()

In [ ]:
#----------------------------------------------------------------------------------------------------------------------------------#
# The PS4 Joystick axes values are mapped to the throttle and steering values that need to be communicated with the Arduino 
# via Serial. axis #2 corresponds to the steering and axis #1 refers to the throttle.
# Important! If you want to make any changes or stop the execution, this cell executes a while loop, you would have to Interrupt 
# the execution of this cell, and then stop the runnig of the camera.
#----------------------------------------------------------------------------------------------------------------------------------#

display(widgets.VBox([image_widget]))

while True:    
    pygame.event.get()         

    joystick = pygame.joystick.Joystick(0)
    joystick.init()
    
    steering_axis = joystick.get_axis(2)
    steering_axis_mapped = mapper(steering_axis, -1, 1, 135, 45)

    throttle_axis = joystick.get_axis(1)   
    
    # there is a break in the range since the inbetween values from 90 to 98 don't result in movement 
    if throttle_axis < 0:
        throttle_axis_mapped = mapper(throttle_axis, -1, 0, 85, 90)
    if throttle_axis > 0:
        throttle_axis_mapped = mapper(throttle_axis, 0, 1, 98, 110)
        
    if throttle_axis == 0:
        throttle_axis_mapped = 92
               
    serial_writer(steering_axis_mapped, throttle_axis_mapped)

    pygame.display.flip()
    clock.tick(20)   

In [ ]:
#----------------------------------------------------------------------------------------------------------------------------------#
# To make any kind of changes to the while loop above, run this after you have interrupted it's execution. 
# Camera has to be stopped first or else everything else goes kaput.
#----------------------------------------------------------------------------------------------------------------------------------#

camera.running = False
camera1_link.unlink()
ser.flush()
pygame.quit()

In [ ]:
#----------------------------------------------------------------------------------------------------------------------------------#
# Experimental code to try and identify if the Joystick disconnects and attempt reconnection. Not tested. Might come in handy.
#----------------------------------------------------------------------------------------------------------------------------------#

# discon = False
# def check_pad():
#     global discon
#     pygame.joystick.quit()
#     pygame.joystick.init()
#     joystick_count = pygame.joystick.get_count()
#     for i in range(joystick_count):
#         joystick = pygame.joystick.Joystick(i)
#         joystick.init()
#     if not joystick_count: 
#         if not discon:
#             print("reconnect you meat bag")
#             discon = True
#         clock.tick(20)
#         check_pad()
#     else:
#         discon = False